# Support Data Insight Analysis

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from dotenv import load_dotenv

import os
import yaml
from crewai import Agent, Task, Crew

In [2]:
load_dotenv()
openai_api_key=os.getenv('OPENAI_API_KEY')

In [3]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'

### Loading Tasks and Agents YAML files

In [6]:
# Define file paths for yaml configurations
files = {
    'agents':'config/agents_4.yaml',
    'tasks':'config/tasks_4.yaml'
}

# Load configurations from yaml files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

### Using FileReadTool

In [7]:
from crewai_tools import FileReadTool

In [8]:
csv_tool = FileReadTool(file_path='./data/support_ticjets_data.csv')

In [9]:
csv_tool

FileReadTool(name="Read a file's content", description="Tool Name: Read a file's content\nTool Arguments: {'file_path': {'description': 'Mandatory file full path to read the file', 'type': 'str'}}\nTool Description: A tool that reads file content. The default file is ./data/support_ticjets_data.csv, but you can provide a different 'file_path' parameter to read another file.", args_schema=<class 'crewai_tools.tools.file_read_tool.file_read_tool.FileReadToolSchema'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x74dde942e7a0>, result_as_answer=False, file_path='./data/support_ticjets_data.csv')

### Creating Agents, Tasks and Crew

In [10]:
# Creating Agents
suggestion_generation_agent = Agent(
  config=agents_config['suggestion_generation_agent'],
  tools=[csv_tool]
)

reporting_agent = Agent(
  config=agents_config['reporting_agent'],
  tools=[csv_tool]
)

chart_generation_agent = Agent(
  config=agents_config['chart_generation_agent'],
  allow_code_execution=False
)


In [11]:
# Creating Tasks
suggestion_generation = Task(
  config=tasks_config['suggestion_generation'],
  agent=suggestion_generation_agent
)

table_generation = Task(
  config=tasks_config['table_generation'],
  agent=reporting_agent
)

chart_generation = Task(
  config=tasks_config['chart_generation'],
  agent=chart_generation_agent
)

final_report_assembly = Task(
  config=tasks_config['final_report_assembly'],
  agent=reporting_agent,
  context=[suggestion_generation, table_generation, chart_generation]
)

In [12]:
# Creating Crew
support_report_crew = Crew(
  agents=[
    suggestion_generation_agent,
    reporting_agent,
    chart_generation_agent
  ],
  tasks=[
    suggestion_generation,
    table_generation,
    chart_generation,
    final_report_assembly
  ],
  verbose=True
)

### Testing our Crew

In [13]:
support_report_crew.test(n_iterations=1, openai_model_name='gpt-4o')

2025-02-12 14:34:37,560 - 128496783281024 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Suggestion Engine
## Task: Generate actionable suggestions for resolving each classified support ticket. The suggestions should be based on: - Issue Type: Tailor suggestions to the specific type of issue reported. - Historical Data: Use historical data such as resolution_time_minutes and
  satisfaction_rating to inform the suggestions.
- Customer Feedback: Incorporate insights from customer_comments to
  customize the suggestions further.

The goal is to provide clear, actionable steps that the support team can take to resolve each issue efficiently and effectively.



# Agent: Suggestion Engine
## Using tool: Read a file's content
## Tool Input: 
"{\"file_path\": \"./data/support_ticjets_data.csv\"}"
## Tool Output: 
Error: File not found at path: ./data/support_ticjets_data.csv


# Agent: Suggestion Engine
## Using tool: Read a file's content
## Tool Input: 
"{\"file_path\": \"./data/support_tickets_data.csv\"}"
## Tool Output: 
ticket_id,customer_id,issue_type,issue_descrip

KeyboardInterrupt: 

### Training your crew and agents

In [ ]:
support_report_crew.train(n_iterations=1, filename='training.pkl')

### Comparing new test result

In [ ]:
support_report_crew.test(n_iterations=1, openai_model_name='gpt-4o')

In [ ]:
# Display the Trello screenshot
from IPython.display import Image, display

# Load and display the image
test_image = Image(filename='test_before_training.png', width=368)
display(test_image)# Display the Trello screenshot
from IPython.display import Image, display

# Load and display the image
test_image = Image(filename='test_before_training.png', width=368)
display(test_image)

### Kicking off Crew

In [ ]:
result = support_report_crew.kickoff()

In [ ]:
from IPython.display import display, Markdown
display(Markdown(result.raw))